In [24]:
import nltk, torch, pickle

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена: 
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ. 
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

In [2]:
class NamesDataset(Dataset):
    def __init__(self, file_path):
        self.special_tokens = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}
        self.word2index = {}
        self.index2word = {}
        self.max_len = 0
        self.data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                self.max_len = max(self.max_len, len(word))
                self.data.append(word)
                for char in word:
                    if char not in self.word2index:
                        index = len(self.word2index)
                        self.word2index[char] = index
                        self.index2word[index] = char
        for token in self.special_tokens:
            index = len(self.word2index)
            self.word2index[token] = index
            self.index2word[index] = token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        word = self.data[index]
        input_seq = [self.word2index['<SOS>']] + [self.word2index.get(char, self.word2index['<UNK>']) for char in word] + [self.word2index['<EOS>']]
        input_seq += [self.word2index['<PAD>']] * (self.max_len - len(input_seq))
        target_seq = input_seq[1:] + [self.word2index['<PAD>']]
        return torch.tensor(input_seq), torch.tensor(target_seq)

In [3]:
dataset = NamesDataset('data/names.txt')
print(len(dataset))
input_seq, target_seq = dataset[17]

input_seq, target_seq

1988


(tensor([32,  0, 18, 19,  7, 21, 21,  7, 22,  9,  0, 33, 30]),
 tensor([ 0, 18, 19,  7, 21, 21,  7, 22,  9,  0, 33, 30, 30]))

In [4]:
dataset[18][1].shape

torch.Size([13])

In [5]:
dataset.word2index["<EOS>"]

33

In [9]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
for input_seq, target_seq in dataloader:
    print(input_seq.shape, target_seq.shape)
    print('...')
    break
print(len(dataloader))

torch.Size([1, 13]) torch.Size([1, 13])
...
1988


In [10]:
next(iter(dataloader))

[tensor([[32,  0,  1,  2,  3,  4,  5,  6, 33, 30, 30, 30, 30]]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6, 33, 30, 30, 30, 30, 30]])]

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

In [11]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        emb = self.embedding(input_seq)
        output, _ = self.rnn(emb)
        output = output.reshape(-1, output.shape[2])
        output = self.linear(output)
        return output

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [12]:
def generate_name(model, dataset, max_len=15, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    with torch.no_grad():
        input_seq = torch.tensor([[dataset.word2index['<SOS>']]])
        name = ''
        while True:
            output = model(input_seq.to(device))
            probs = nn.functional.softmax(output[-1], dim=0)
            next_token = torch.multinomial(probs, num_samples=1).item()
            if next_token == dataset.word2index['<EOS>'] or len(name) >= max_len:
                break
            name += dataset.index2word[next_token]
            input_seq = torch.tensor([[next_token]])
        return name

In [13]:
def train(model, dataloader, dataset, num_epochs, lr, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.word2index['<PAD>'])
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(num_epochs):
        total_loss = 0
        for input_seq, target_seq in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            loss = criterion(output, target_seq.reshape(-1))

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.5f}')

        if (epoch + 1) % 5 == 0:
            print(f"Предсказанные имена: {[generate_name(model, dataset) for _ in range(4)]}")

In [14]:
model = RNNModel(len(dataset.word2index), 16, 32)
train(model, dataloader, dataset, 50, 0.04)

Epoch 1/50: 100%|██████████| 1988/1988 [00:07<00:00, 283.46it/s]


Epoch 1/50, Loss: 3.20870


Epoch 2/50: 100%|██████████| 1988/1988 [00:04<00:00, 421.68it/s]


Epoch 2/50, Loss: 3.36745


Epoch 3/50: 100%|██████████| 1988/1988 [00:04<00:00, 434.74it/s]


...

Epoch 48/50, Loss: 3.47270


Epoch 49/50: 100%|██████████| 1988/1988 [00:04<00:00, 453.77it/s]


Epoch 49/50, Loss: 3.46810


Epoch 50/50: 100%|██████████| 1988/1988 [00:04<00:00, 450.80it/s]

Epoch 50/50, Loss: 3.50346
Предсказанные имена: ['яняняняняняня', 'яняняняняняняша', 'яняня', 'яняняняняхрянян']


In [23]:
generate_name(model, dataset)

'яняша'

In [25]:
# Сохранение модели в файл
with open("rnn_name_generator.pickle", "wb") as f:
    pickle.dump(model, f)

In [26]:
# Загрузка модели из файла
with open("rnn_name_generator.pickle", "rb") as f:
    model = pickle.load(f)

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;


In [43]:
import string

with open('data/prestuplenie-i-nakazanie.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Remove newlines and extra spaces
text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
text = ' '.join(text.split())

seq_len = 100
step = 5

sentences = []
next_chars = []
for i in range(0, len(text) - seq_len, step):
    sentences.append(text[i:i+seq_len])
    next_chars.append(text[i+seq_len])

print('Number of sequences:', len(sentences))

Number of sequences: 215480


In [44]:
# Create char to index and index to char dictionaries
chars = sorted(list(set(text)))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

In [45]:
class TextDataset(Dataset):
    def __init__(self, text, seq_len, step):
        self.seq_len = seq_len
        self.step = step

        self.sentences = []
        self.next_chars = []
        for i in range(0, len(text) - seq_len, step):
            self.sentences.append(text[i:i+seq_len])
            self.next_chars.append(text[i+seq_len])

        self.char_to_index = {c: i for i, c in enumerate(sorted(list(set(text))))}

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        x = [self.char_to_index[c] for c in self.sentences[idx]]
        y = self.char_to_index[self.next_chars[idx]]
        return torch.LongTensor(x), torch.LongTensor([y])

    @property
    def vocab_size(self):
        return len(self.char_to_index)

In [46]:
dataset = TextDataset(text, seq_len=30, step=5)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
dataset.vocab_size

142

In [47]:
class TextGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.5):
        super(TextGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        embedded = self.embedding(x)
        out, h = self.lstm(embedded, h)
        out = self.fc(out[:, -1, :])
        return out, h

    def init_hidden(self, batch_size, device):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device))

input_size = len(string.printable)
hidden_size = 256
output_size = len(string.printable)

model = TextGenerator(input_size, hidden_size, output_size, num_layers=2, dropout=0.5)
model.to("cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

In [48]:
def train(model, dataloader, dataset, num_epochs, lr, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(num_epochs):
        total_loss = 0
        model.train()
        for x, y in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            hidden = model.init_hidden(x.size(0), device)
            output, hidden = model(x, hidden)

            loss = criterion(output.view(-1, dataset.vocab_size), y.view(-1))
            loss.backward()

            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)

            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

        if (epoch+1) % 10 == 0:
            model.eval()
            generated_text = generate_text(model, 'преступление', 100, device=device)
            print(f"Generated text:\n{generated_text}")

In [49]:
index_to_char

{0: ' ',
 1: '!',
 2: '(',
 3: ')',
 4: '*',
 5: ',',
 6: '-',
 7: '.',
 8: '0',
 9: '1',
 10: '2',
 11: '3',
 12: '4',
 13: '5',
 14: '6',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: ';',
 20: '=',
 21: '?',
 22: 'A',
 23: 'B',
 24: 'C',
 25: 'D',
 26: 'G',
 27: 'I',
 28: 'M',
 29: 'N',
 30: 'O',
 31: 'P',
 32: 'T',
 33: 'V',
 34: 'X',
 35: '[',
 36: ']',
 37: 'a',
 38: 'b',
 39: 'c',
 40: 'd',
 41: 'e',
 42: 'f',
 43: 'g',
 44: 'h',
 45: 'i',
 46: 'j',
 47: 'k',
 48: 'l',
 49: 'm',
 50: 'n',
 51: 'o',
 52: 'p',
 53: 'q',
 54: 'r',
 55: 's',
 56: 't',
 57: 'u',
 58: 'v',
 59: 'w',
 60: 'x',
 61: 'y',
 62: 'z',
 63: '«',
 64: '°',
 65: '»',
 66: 'à',
 67: 'ä',
 68: 'è',
 69: 'é',
 70: 'ê',
 71: 'ó',
 72: 'ö',
 73: 'ü',
 74: 'ł',
 75: 'А',
 76: 'Б',
 77: 'В',
 78: 'Г',
 79: 'Д',
 80: 'Е',
 81: 'Ж',
 82: 'З',
 83: 'И',
 84: 'К',
 85: 'Л',
 86: 'М',
 87: 'Н',
 88: 'О',
 89: 'П',
 90: 'Р',
 91: 'С',
 92: 'Т',
 93: 'У',
 94: 'Ф',
 95: 'Х',
 96: 'Ц',
 97: 'Ч',
 98: 'Ш',
 99: 'Щ',
 100: 'Э',

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3

In [50]:
def generate_text(model, prime, length, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    model.to(device)
    model.eval()
    hidden = model.init_hidden(1, device=device)

    prime = [char_to_index[c] for c in prime]
    prime = torch.LongTensor(prime).unsqueeze(0).to(device)

    _, hidden = model(prime, hidden)
    generated = prime

    for i in range(length):
        outputs, hidden = model(generated[:, -seq_len:], hidden)
        _, topi = outputs.topk(1)
        generated = torch.cat((generated, topi), dim=1)

    generated = generated.squeeze().tolist()
    text = ''.join([index_to_char[i] for i in generated])

    return text

In [51]:
model = TextGenerator(input_size=dataset.vocab_size, hidden_size=64, output_size=dataset.vocab_size, num_layers=2, dropout=0.5)
train(model, dataloader, dataset, num_epochs=10, lr=0.003)

Epoch 1/10: 100%|██████████| 1684/1684 [02:36<00:00, 10.79it/s]


Epoch 1/10, Loss: 2.6145


Epoch 2/10: 100%|██████████| 1684/1684 [02:27<00:00, 11.40it/s]


Epoch 2/10, Loss: 2.3219


Epoch 3/10: 100%|██████████| 1684/1684 [02:33<00:00, 10.94it/s]


...

Epoch 8/10, Loss: 2.0518


Epoch 9/10: 100%|██████████| 1684/1684 [02:40<00:00, 10.50it/s]


Epoch 9/10, Loss: 2.0359


Epoch 10/10: 100%|██████████| 1684/1684 [02:34<00:00, 10.89it/s]


Epoch 10/10, Loss: 2.0221
Generated text:
преступление и вы вы вы в только в только в только в только в только в только в только в только в только в тольк


In [52]:
train(model, dataloader, dataset, num_epochs=50, lr=0.003)

Epoch 1/50: 100%|██████████| 1684/1684 [02:27<00:00, 11.44it/s]


Epoch 1/50, Loss: 2.0135


Epoch 2/50: 100%|██████████| 1684/1684 [02:27<00:00, 11.42it/s]


Epoch 2/50, Loss: 2.0027


Epoch 3/50: 100%|██████████| 1684/1684 [02:27<00:00, 11.39it/s]


...

Epoch 28/50, Loss: 1.8977


Epoch 29/50: 100%|██████████| 1684/1684 [02:18<00:00, 12.17it/s]


Epoch 29/50, Loss: 1.8975


Epoch 30/50:  60%|█████▉    | 1005/1684 [01:23<00:56, 11.99it/s]


KeyboardInterrupt: 

In [53]:
prime = 'Раскольников открывает дверь'
length = 100

generated_text = generate_text(model, prime, length)
print(generated_text)

Раскольников открывает дверь в собенно и не в собенно и не в собенно и не в собенно и не в собенно и не в собенно и не в собенно
